In [11]:
import pandas as pd
import numpy as np

# Analisis de trayectorio

Dataset de la informacion de paraderos (busstop.csv):
- Muestra informacion de los paraderos que se utilizan durante el trayectorio de los autobuses, tanto como la latitud y longitud.
- Se consideran puntos gps en formato de latitud y longitud


Se procesa la nueva trayectoria de paraderos y analizar los radios que se les asignaran para el geofencing, creando un nuevo archivo de la trayectoria de paraderos.

In [13]:
ida = pd.read_csv('../data/busstops_ida.csv', sep=',')
ida = ida.drop(columns=['radio', 'number'])
ida['tipo'] = ['ida' for _ in range(len(ida))]
#ida

In [14]:
vuelta =  pd.read_csv('../data/busstop.csv', sep=',')
vuelta = vuelta.drop(columns=['number', 'radio'])
vuelta = vuelta.tail(152-81)
vuelta['tipo'] = ['vuelta' for _ in range(len(vuelta))]
#vuelta

In [77]:
allbusstop_df = pd.concat([ida, vuelta], ignore_index=True)
allbusstop_df['id'] = [f'{i}' for i in range(len(allbusstop_df))]
print(allbusstop_df.info())
allbusstop_df['latitude_longitude'] = allbusstop_df.apply(lambda row: (row['latitude'], row['longitude']), axis=1)
allbusstop_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          155 non-null    object 
 1   name        155 non-null    object 
 2   navigation  155 non-null    int64  
 3   latitude    155 non-null    float64
 4   longitude   155 non-null    float64
 5   tipo        155 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 7.4+ KB
None


,id,name,navigation,latitude,longitude,tipo,latitude_longitude
0,0,inicio,159,-13.554978,-71.856025,ida,"(-13.5549784, -71.856025)"
1,1,primer stop,304,-13.554812,-71.857300,ida,"(-13.5548123, -71.8573001)"
2,2,segundo stop,281,-13.553294,-71.861944,ida,"(-13.5532944, -71.8619442)"
3,3,Ccollana,283,-13.552296,-71.867299,ida,"(-13.552296, -71.867299)"
4,4,Puente Huaccoto,283,-13.551592,-71.870211,ida,"(-13.551592, -71.870211)"
...,...,...,...,...,...,...,...
150,150,Tapia,107,-13.549241,-71.878975,vuelta,"(-13.549241, -71.878975)"
151,151,Forestal,107,-13.549985,-71.876345,vuelta,"(-13.549985, -71.876345)"
152,152,Kayra,107,-13.550608,-71.874402,vuelta,"(-13.550608, -71.874402)"
153,153,Puente,105,-13.551644,-71.870166,vuelta,"(-13.551644, -71.870166)"


In [5]:
# Guarda el DataFrame modificado en un nuevo archivo CSV
# allbusstop_df.to_csv('../data/allbusstop.csv', index=False)

Generar los links de ida y vuelta

In [16]:
links_df = pd.DataFrame()
ini = allbusstop_df['id'].iloc[:-1].values
end = allbusstop_df['id'].iloc[1:].values
links_df['link'] = [f'{i}:{e}' for i, e in zip(ini, end )]
links_df

,link
0,0:1
1,1:2
2,2:3
3,3:4
4,4:5
...,...
149,149:150
150,150:151
151,151:152
152,152:153


a
<img src='graficos/trayectoria_bus_stops.png' width='600' height='300'>

# 2. Analisis de los registros de monitorizacion

Tareas a realizar:
- Filtrar los paraderos de vuelta (los que se usa y eliminar los que no se usan y que conincidan con el de ida)

Se da nuevo formato a la tabla, creando nuevas columnas y eliminando otras.

In [17]:
# cargar datos en bruto
monitoringdf = pd.read_csv('../data/_Process_monitoring__202303251137.csv', sep=',')
monitoringdf = monitoringdf[['id', 'latitude','longitude','date','time', 'lap', 'navigation', 'vehicle_id']]
monitoringdf

,id,latitude,longitude,date,time,lap,navigation,vehicle_id
0,13173398,-13.549597,-71.877510,2023-03-01,05:10:11,1,285,SJ39
1,13177467,-13.524529,-71.977527,2023-03-01,06:04:51,1,298,SJ34
2,13189674,-13.534183,-71.980709,2023-03-01,07:22:21,1,7,SJ29
3,13192729,-13.551146,-71.986928,2023-03-01,07:39:04,1,202,SJ21
4,13193272,-13.548642,-71.881010,2023-03-01,07:41:52,1,107,SJ34
...,...,...,...,...,...,...,...,...
1685065,17241845,-13.525440,-71.975079,2023-03-25,08:50:24,2,288,SJ24
1685066,17253753,-13.551121,-71.989621,2023-03-25,10:01:56,2,253,SJ16
1685067,17257644,-13.543207,-71.897785,2023-03-25,10:26:10,2,126,SJ24
1685068,17266468,-13.532774,-71.921865,2023-03-25,11:14:26,1,258,SJ43


# Cleaning

Mostramos la cantidad de datos, existen puntos que por error de lectura se leen dos veces, estos puntos se detectan si en las columnas datetime y latitud-longitud existen datos iguales, esto no debe suceder ya que el bus esta en constante movimiento como el tiempo. Como consecuencia se decide eliminar estos duplicados.

In [18]:
# DUPLICADOS
print(f'Shape dataset: {monitoringdf.shape}')

# Verificar y contar si hay puntos duplicados segun datetime y latitude_longitude
base = ['date', 'time', 'latitude', 'longitude']
twocols_dupli = monitoringdf.duplicated(subset=base)
print(f"Nro de duplicados en {base}: {twocols_dupli.sum()}")

# ============================ Solo para comprobacion ============================
# Verificar y contar si hay puntos duplicados segun latitude_longitude
# onecol_dupli =  monitoringdf['latitude_longitude'].duplicated()
# print(len(twocols_dupli), len(onecol_dupli))
# print(twocols_dupli.sum(), onecol_dupli.sum())
# print(abs(twocols_dupli.sum() - onecol_dupli.sum()))
# tmp = [ not x==y for x,y in zip(twocols_dupli, onecol_dupli)]
# data_tmp = monitoringdf.copy()
# data_tmp['dupli'] = tmp
# data_tmp[data_tmp['dupli'] == True]
# ================================================================================

# Eliminar columnas
monitoringdf.drop_duplicates(subset=base, inplace=True)
print(f"Duplicados eliminados => shape dataset: {monitoringdf.shape}")

Shape dataset: (1685070, 8)
Nro de duplicados en ['date', 'time', 'latitude', 'longitude']: 13116
Duplicados eliminados => shape dataset: (1671954, 8)


Comprobamos si existen casillas vacias (Nans) en todo el dataframe.

In [19]:
# NANs
# verificar si hay datos faltantes
print("Cantidad de NaNs:")
for col in monitoringdf:
    nans = monitoringdf[col].isna().sum()
    print(f"\tColumna {col}: {nans}")

Cantidad de NaNs:
	Columna id: 0
	Columna latitude: 0
	Columna longitude: 0
	Columna date: 0
	Columna time: 0
	Columna lap: 0
	Columna navigation: 0
	Columna vehicle_id: 0


Muestra es una ***ida y vuelta de un vehiculo***

In [20]:
# preparar el dataset para iterar en todos los posibles samples
samples_df = monitoringdf.copy()

# Crear columna sample para identificar los samples, utilizando la fecha vehiculo y lapso, todo ello encapsulado en un string
samples_df['sample'] = samples_df.apply(lambda row: f"{row['date']} {row['vehicle_id']} {row['lap']}" , axis=1)
samples_dict = {k:v for v, k in enumerate(samples_df['sample'].unique())}
samples_df['sample'] = samples_df['sample'].apply(lambda sample: samples_dict[sample])
samples_df.sort_values(by='sample', inplace=True)
print('nro posibles samples:', len(samples_dict))

# convertir date and time en el formato datetime
samples_df['datetime'] = samples_df['date'] + ' ' + samples_df['time']
samples_df['datetime'] = pd.to_datetime(samples_df['datetime'], format='%Y-%m-%d %H:%M:%S')
samples_df.drop(columns=['date', 'time'], inplace=True)

samples_df

nro posibles samples: 5015


,id,latitude,longitude,lap,navigation,vehicle_id,sample,datetime
104831,13181518,-13.522822,-71.969916,1,102,SJ39,0,2023-03-01 06:35:15
49342,13179415,-13.531396,-71.979872,1,12,SJ39,0,2023-03-01 06:20:32
27531,13178576,-13.545698,-71.983907,1,42,SJ39,0,2023-03-01 06:13:46
57321,13175650,-13.525765,-71.982937,1,111,SJ39,0,2023-03-01 05:44:47
25378,13174422,-13.530906,-71.929595,1,288,SJ39,0,2023-03-01 05:24:51
...,...,...,...,...,...,...,...,...
1679049,17019979,-13.553088,-71.863690,0,98,SJ32,5014,2023-03-24 05:13:08
1676807,17019886,-13.551734,-71.869753,0,101,SJ32,5014,2023-03-24 05:11:33
1677151,17019900,-13.551959,-71.868857,0,106,SJ32,5014,2023-03-24 05:11:45
1675441,17019830,-13.550220,-71.873981,0,194,SJ32,5014,2023-03-24 05:10:23


# Analisis de Samples

In [22]:
# Funcion para extraer un sample por id o nombre y csv para graficar
def getSample(id=-1, name='', to_csv=False):
    if(name != ''):
        id = samples_dict.get(name, -1)
    
    if (id != -1):
        tmp = samples_df[samples_df['sample'] == id].sort_values(by='datetime').reset_index(drop=True)
        print(tmp.shape)
        name = f"{str(tmp.loc[0,'datetime']).split()[0]} {tmp.loc[0,'vehicle_id']} {tmp.loc[0,'lap']}"
        print(tmp.to_string())
        if to_csv: 
            tmp.to_csv(f'{name}- {id} -sample.csv', index=False)
    else:
        print(f'No existe una muestra {id} o {name}')

In [23]:
# cantidad de laps y vehiculos
veh = samples_df['vehicle_id'].unique()
lap = samples_df['lap'].unique()

print(len(veh))
print(sorted(veh))

print(len(lap))
print(sorted(lap))

43
['SJ01', 'SJ02', 'SJ03', 'SJ04', 'SJ05', 'SJ06', 'SJ07', 'SJ08', 'SJ09', 'SJ10', 'SJ11', 'SJ12', 'SJ13', 'SJ14', 'SJ15', 'SJ16', 'SJ17', 'SJ18', 'SJ19', 'SJ20', 'SJ21', 'SJ22', 'SJ23', 'SJ24', 'SJ25', 'SJ26', 'SJ27', 'SJ28', 'SJ29', 'SJ30', 'SJ31', 'SJ32', 'SJ33', 'SJ34', 'SJ35', 'SJ36', 'SJ37', 'SJ38', 'SJ39', 'SJ40', 'SJ41', 'SJ42', 'SJ43']
9
[np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8)]


In [24]:
# cantidad de puntos por LAP
samples_df.groupby('lap').size()

lap
0     43941
1    352566
2    352893
3    342559
4    326027
5    207131
6     40302
7      5896
8       639
dtype: int64

In [25]:
# Nro de samples por LAP
cp = []
for lap in range(9):
    v = samples_df[samples_df['lap'] == lap]['sample'].unique().shape[0]
    cp.append(v)

# lap = indice de la lista
cp

[932, 911, 900, 847, 768, 528, 110, 17, 2]

In [26]:
# id de samples por LAP
samples_df[samples_df['lap'] == 7]['sample'].unique()

array([ 363,  904, 1031, 1258, 1795, 1820, 2150, 2817, 3073, 3234, 3305,
       3883, 3926, 3927, 4257, 4297, 4389])

In [27]:
samples_sizes = samples_df.groupby('sample').size().to_list()
print('Max:', max(samples_sizes))
print('Min:', min(samples_sizes))

# cantidad de samples por nro de puntos
count, bins = np.histogram(samples_sizes, bins=10, range=(0, 2400))
bins = np.round(bins, 0)
ranges = {f'{int(bins[i])}-{int(bins[i+1])-1}':int(count[i]) for i in range(len(count))}
ranges

Max: 2328
Min: 1


{'0-239': 1391,
 '240-479': 3125,
 '480-719': 390,
 '720-959': 97,
 '960-1199': 8,
 '1200-1439': 2,
 '1440-1679': 1,
 '1680-1919': 0,
 '1920-2159': 0,
 '2160-2399': 1}

Se cuenta con 155 paraderos por lo que el minimo de registro que debemos aceptar es de 155

In [28]:
lap0_sizes = samples_df[samples_df['lap'] == 0].groupby('sample').size().to_list()
c, b = np.histogram(lap0_sizes, bins=10)
b = np.round(b, 0)
r = {f'{int(b[i])}-{int(b[i+1])-1}':int(c[i]) for i in range(len(c))}
print('max:', max(lap0_sizes))
print('min:', min(lap0_sizes))
r

max: 446
min: 1


{'1-45': 674,
 '46-89': 103,
 '90-133': 73,
 '134-178': 46,
 '179-223': 20,
 '224-267': 7,
 '268-311': 4,
 '312-356': 1,
 '357-401': 1,
 '402-445': 3}

En base al analisis se determina que lap 0 no posee datos que ayuden en la prediccion por lo que seran eliminados

In [29]:
samples_dfp = samples_df[(samples_df['lap'] != 0) & (samples_df['lap'] != 8) ].copy()
samples_dfp.shape

(1627374, 8)

In [30]:
samples_dfp['lap'].unique()

array([1, 2, 3, 4, 5, 6, 7])

In [32]:
sizes = samples_dfp.groupby('sample').size()
max(sizes), min(sizes)

# Nro de samples por LAP
ids_mins = []
for l in range(1, 8):
    v = samples_dfp[samples_dfp['lap'] == l]['sample'].unique()
    sizes = samples_dfp[samples_dfp['lap'] == l].groupby('sample').size().to_dict()
    print('lap:' ,l, len(v), max(sizes), min(sizes))
    ids_mins.append(sizes)

lap: 1 911 4943 0
lap: 2 900 4960 5
lap: 3 847 4988 12
lap: 4 768 4965 14
lap: 5 528 5008 18
lap: 6 110 5012 172
lap: 7 17 4389 363


In [33]:
samples_dfp.groupby('sample').size().to_dict()

{0: 427,
 1: 433,
 2: 291,
 3: 451,
 4: 426,
 5: 447,
 6: 467,
 7: 454,
 8: 262,
 9: 397,
 10: 401,
 11: 430,
 12: 481,
 13: 470,
 14: 444,
 15: 434,
 16: 450,
 17: 465,
 18: 402,
 19: 468,
 20: 464,
 22: 443,
 23: 420,
 24: 125,
 25: 331,
 26: 431,
 27: 446,
 28: 449,
 29: 450,
 30: 458,
 31: 427,
 32: 447,
 33: 415,
 34: 426,
 35: 450,
 36: 435,
 37: 475,
 38: 449,
 39: 357,
 40: 439,
 41: 453,
 42: 402,
 43: 404,
 44: 415,
 45: 452,
 46: 427,
 47: 437,
 48: 480,
 49: 272,
 50: 451,
 51: 125,
 52: 263,
 53: 414,
 54: 420,
 55: 454,
 56: 480,
 57: 429,
 58: 899,
 59: 450,
 60: 427,
 61: 446,
 62: 433,
 63: 438,
 64: 501,
 65: 433,
 66: 467,
 67: 467,
 68: 429,
 69: 457,
 70: 464,
 71: 458,
 72: 443,
 73: 123,
 74: 406,
 75: 525,
 76: 436,
 77: 684,
 78: 454,
 79: 448,
 80: 459,
 81: 411,
 82: 438,
 83: 495,
 84: 272,
 86: 414,
 87: 465,
 88: 433,
 89: 457,
 90: 443,
 91: 412,
 92: 444,
 93: 370,
 94: 467,
 95: 456,
 96: 429,
 97: 270,
 98: 622,
 99: 418,
 100: 432,
 101: 388,
 102: 42

In [34]:
fff = samples_dfp.groupby('sample').size().to_list()
print('max', max(fff))
print('min', min(fff))
ccc, bbb = np.histogram(fff, bins=1, range=(0, 220))
bbb = np.round(bbb, 0)
{f'{int(bbb[i])}-{int(bbb[i+1])-1}':int(ccc[i]) for i in range(len(ccc))}

max 2328
min 1


{'0-219': 434}

In [36]:
d = samples_dfp.groupby('sample').size().to_dict()
ids_to_drop = {}
for k, v in d.items():
    if v <= 220:
        ids_to_drop[k]=v
        
len(ids_to_drop), ids_to_drop

(434,
 {24: 125,
  51: 125,
  73: 123,
  153: 14,
  160: 189,
  161: 145,
  182: 38,
  194: 165,
  198: 13,
  217: 220,
  261: 77,
  265: 3,
  283: 1,
  288: 171,
  304: 8,
  320: 42,
  337: 1,
  462: 149,
  500: 214,
  517: 101,
  526: 141,
  527: 125,
  530: 179,
  550: 18,
  552: 62,
  558: 139,
  593: 122,
  621: 171,
  626: 55,
  630: 22,
  632: 3,
  635: 29,
  653: 93,
  673: 210,
  676: 154,
  682: 77,
  691: 36,
  695: 35,
  705: 181,
  707: 1,
  710: 43,
  716: 93,
  723: 45,
  731: 146,
  734: 22,
  743: 1,
  745: 24,
  746: 15,
  759: 9,
  764: 6,
  776: 31,
  780: 128,
  788: 183,
  793: 9,
  800: 1,
  801: 125,
  802: 3,
  805: 14,
  810: 24,
  814: 8,
  820: 1,
  824: 1,
  840: 207,
  894: 109,
  929: 198,
  937: 132,
  944: 187,
  957: 198,
  980: 212,
  1031: 168,
  1051: 125,
  1060: 218,
  1077: 218,
  1078: 32,
  1080: 170,
  1085: 20,
  1098: 3,
  1106: 133,
  1137: 102,
  1138: 179,
  1143: 14,
  1145: 20,
  1155: 32,
  1184: 219,
  1195: 96,
  1211: 3,
  1233: 11,

In [37]:
getSample(id=1060, to_csv=True)

(218, 8)
           id   latitude  longitude  lap  navigation vehicle_id  sample            datetime
0    13911376 -13.551356 -71.988164    2         286       SJ20    1060 2023-03-05 09:13:13
1    13911410 -13.551301 -71.988999    2         304       SJ20    1060 2023-03-05 09:13:35
2    13911431 -13.551319 -71.989830    2         233       SJ20    1060 2023-03-05 09:13:47
3    13911459 -13.551822 -71.990522    2         232       SJ20    1060 2023-03-05 09:14:02
4    13911485 -13.552126 -71.991177    2         278       SJ20    1060 2023-03-05 09:14:19
5    13911533 -13.551687 -71.990891    2          54       SJ20    1060 2023-03-05 09:14:43
6    13911549 -13.551224 -71.990037    2          57       SJ20    1060 2023-03-05 09:14:56
7    13911576 -13.550732 -71.989243    2          55       SJ20    1060 2023-03-05 09:15:14
8    13911600 -13.550303 -71.988744    2          46       SJ20    1060 2023-03-05 09:15:26
9    13911622 -13.549774 -71.988116    2          48       SJ20    1060

Analizando las graficas, todos los samples con menos de 220 puntos no registran el circuito completo

# Funciones para calcular el travel time

In [81]:
from haversine import haversine as hvs, Unit
import numpy as np

# FUNCION: 
def searchStopsPoints(row, stops, r_nav=45, r_dtec=60):
    # verificar la navegacion y el radio de distancia
    stops = stops[abs(stops['navigation'] - row['navigation']) <= r_nav]
    stops['dis'] = stops['latitude_longitude'].apply(lambda stop: round(hvs(row['latitude_longitude'], stop, unit=Unit.METERS), 2))
    stops = stops[stops['dis'] <= r_dtec]
    
    # preparar el output
    row['stop'] = str(stops.loc[stops['dis'].idxmin(), 'id']) if len(stops) != 0 else np.nan
    row['dis'] = float(stops.loc[stops['dis'].idxmin(), 'dis']) if len(stops) != 0 else 0    
    #row['long_lati'] = row['latitude_longitude']
    return row

# FUNCION: CALCULA EL TIEMPO EN SEG DE UN PUNTO CON SU ANTERIOR
def time_travel(row, df):
    if {row['init_stop'], row['end_stop']}.issubset(set(df.index)):
        # print('if')
        ti, te =  df.loc[row['init_stop'], 'datetime'], df.loc[row['end_stop'], 'datetime']
        row['date_time_init'] = ti
        row['date_time_end'] = te
        time = abs((ti - te).total_seconds())
        
        #row['time_travel'] =  time if time < 300 else np.nan
        row['time_travel'] =  time
        row['init_idp'], row['end_idp'] = df.loc[row['init_stop'], 'id'], df.loc[row['end_stop'], 'id']

        row['ini_lat'] = df.loc[row['init_stop'], 'latitude_longitude']
        row['end_lat'] = df.loc[row['end_stop'], 'latitude_longitude']

        row['veh'] = df.loc[row['init_stop'], 'vehicle_id']
        row['lap'] = df.loc[row['init_stop'], 'lap']
        row['date'] = df.loc[row['init_stop'], 'date']


    else:
        # print('else')
        row['date_time_init'] = np.nan
        row['date_time_end'] = np.nan
        row['time_travel'] = np.nan
        row['init_idp'] = np.nan
        row['end_idp'] = np.nan
        row['veh'] = np.nan
        row['lap'] = np.nan
        row['date'] = np.nan
        
    return row

# Crear secuencias y calcular tiempos de viaje

In [41]:
index_max = samples_sizes.index(2328)
samples_df[samples_df['sample'] == index_max]

,id,latitude,longitude,lap,navigation,vehicle_id,sample,datetime
1221192,16266861,-13.524031,-71.956814,1,287,SJ01,3720,2023-03-20 16:14:53
1230461,16267078,-13.540599,-71.981332,1,8,SJ01,3720,2023-03-20 17:07:17
1359362,16266088,-13.537926,-71.904708,1,302,SJ01,3720,2023-03-20 13:04:14
1259914,16267758,-13.538938,-71.980852,1,198,SJ01,3720,2023-03-20 19:30:58
1359318,16266087,-13.538442,-71.903886,1,304,SJ01,3720,2023-03-20 13:04:02
...,...,...,...,...,...,...,...,...
1259874,16263688,-13.551513,-71.990616,1,209,SJ01,3720,2023-03-20 08:18:53
1259990,16263691,-13.551968,-71.991290,1,208,SJ01,3720,2023-03-20 08:19:09
1301444,16264684,-13.527570,-71.943359,1,275,SJ01,3720,2023-03-20 10:37:12
1259954,16267759,-13.539696,-71.981144,1,202,SJ01,3720,2023-03-20 19:31:12


In [88]:
metrics_df = pd.DataFrame(columns=['id', 'sample'])

c = 0
for id, sample_df in samples_df.groupby('sample'):
    name = f'{str(sample_df.iloc[0, 6]).split()[0]} {sample_df.iloc[0, 5]} {sample_df.iloc[0, 3]}'
    if(sample_df.shape[0] < 5):
        print(name, f'- {sample_df.shape}')
    c += 1
c

166 SJ41 0 - (4, 8)
262 SJ25 0 - (1, 8)
265 SJ40 5 - (3, 8)
283 SJ08 5 - (1, 8)
298 SJ42 0 - (2, 8)
330 SJ17 0 - (1, 8)
335 SJ09 0 - (1, 8)
337 SJ07 5 - (1, 8)
532 SJ43 0 - (3, 8)
632 SJ43 5 - (3, 8)
633 SJ35 0 - (1, 8)
707 SJ26 5 - (1, 8)
741 SJ29 0 - (1, 8)
743 SJ40 4 - (1, 8)
762 SJ42 0 - (1, 8)
770 SJ33 0 - (2, 8)
774 SJ43 0 - (1, 8)
800 SJ06 5 - (1, 8)
802 SJ08 3 - (3, 8)
807 SJ19 0 - (4, 8)
820 SJ25 6 - (1, 8)
824 SJ34 5 - (1, 8)
868 SJ35 0 - (4, 8)
1098 SJ36 1 - (3, 8)
1115 SJ13 0 - (3, 8)
1191 SJ24 0 - (3, 8)
1211 SJ42 6 - (3, 8)
1214 SJ21 0 - (2, 8)
1232 SJ09 0 - (4, 8)
1239 SJ19 5 - (2, 8)
1241 SJ19 0 - (2, 8)
1242 SJ41 0 - (4, 8)
1258 SJ08 7 - (1, 8)
1286 SJ01 0 - (2, 8)
1293 SJ34 6 - (1, 8)
1295 SJ21 0 - (1, 8)
1301 SJ15 5 - (1, 8)
1474 SJ10 6 - (4, 8)
1548 SJ33 0 - (3, 8)
1644 SJ25 6 - (1, 8)
1657 SJ18 5 - (1, 8)
1659 SJ05 5 - (1, 8)
1662 SJ29 0 - (2, 8)
1689 SJ11 0 - (1, 8)
1691 SJ39 0 - (2, 8)
1704 SJ18 1 - (1, 8)
1714 SJ26 0 - (3, 8)
1730 SJ29 0 - (1, 8)
2005 SJ43 1 - (

5015

In [89]:
samples_df.shape

(1671954, 8)

In [90]:
samples_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1671954 entries, 104831 to 1675685
Data columns (total 8 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   id          1671954 non-null  int64         
 1   latitude    1671954 non-null  float64       
 2   longitude   1671954 non-null  float64       
 3   lap         1671954 non-null  int64         
 4   navigation  1671954 non-null  int64         
 5   vehicle_id  1671954 non-null  object        
 6   sample      1671954 non-null  int64         
 7   datetime    1671954 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 114.8+ MB


In [91]:
sample_df_tmp = samples_df.copy()
sample_df_tmp['date'] = sample_df_tmp['datetime'].apply(lambda dt: str(dt).split(' ')[0])
sample_df_tmp['latitude_longitude'] = sample_df_tmp.apply(lambda row: (row['latitude'], row['longitude']), axis=1)
sample_df_tmp.shape

(1671954, 10)

In [92]:
# iterar en todos los posibles
n_descartados = 0
list_err = []
list_samplesdf = []
date = ''
bt = 0

# Se ordena los posibles samples de manera ascendente
for ids, pointsdf in sample_df_tmp.groupby('sample'): 

    # Seguir procesamiento
    if date != pointsdf['date'].unique()[0]:
        if (bt == 1):
            break
        bt+=1

        date = pointsdf['date'].unique()[0]
        print(date)

    # ordenar la secuencia por datetime
    pointsdf.sort_values(by='datetime', inplace=True)
    
    stopsPointsdf = pointsdf.apply(lambda row: searchStopsPoints(row, allbusstop_df), axis=1)

    stopsPointsdf = stopsPointsdf[stopsPointsdf['stop'].notna()].reset_index()

    # Seleccionar un unico punto por stops
    stopsPointsdf.sort_values(by=['stop', 'dis'], ascending=True, inplace=True, ignore_index=True, key=np.int64)
    stopsPointsdf.drop_duplicates(subset=['stop'], keep='first', inplace=True, ignore_index=True)

    # filtrar los stop sin puntos 
    missing_stops = list(set(allbusstop_df['id']) - set(stopsPointsdf['stop'].unique()))

    # verificar la cantidad de puntos filtrados
    if len(missing_stops) < 24:
        # verificar la secuencialidad de los datetime
        byS = list(stopsPointsdf.sort_values(by='stop')['stop'])
        byT = list(stopsPointsdf.sort_values(by='datetime')['stop'])
        if byS == byT:
            # crear sample y calcular el travel_time
            sampledf = links_df.copy()
            stopsPointsdf.set_index('stop', inplace=True)
            sampledf = sampledf.apply(lambda row: time_travel(row, stopsPointsdf), axis=1)
            
            # crear el id_sample
            list_samplesdf.append(sampledf)
        else:
            # registrar sample defectoso
            list_err.append(stopsPointsdf)
            print(' ', ids)
    else:
        n_descartados += 1

# mostrar nro de samples encontrados
print('='*100)
print(f"{len(list_samplesdf)} Samples buenos")
print(f"{len(list_err)} Samples con errores de continuidad de tiempo")
print(f"{n_descartados} Samples descartados por falta de puntos")

2023-03-01
  0
  1
  3
  4
  5
  6
  7
  9
  10
  11
  12
  13
  14
  15
  16
  17
  18
  19
  20
  22
  23
  26
  27
  28
  29
  30
  31
  32
  33
  34
  35
  36
  37
  38
  40
  41
  42
  43
  45
  46
  47
  48
  50
  53
  54
  55
  56
  57
  58
  59
  60
  61
  62
  63
  64
  65
  66
  67
  68
  69
  70
  71
  72
  74
  75
  76
  77
  78
  79
  80
  81
  82
  83
  86
  87
  88
  89
  90
  92
  94
  95
  96
  98
  99
  100
  101
  102
  103
  104
  105
  106
  107
  108
  110
  111
  112
  113
  114
  115
  116
  117
  118
  122
  123
  124
  126
  127
  128
  129
  131
  132
  133
  134
  135
  136
  137
  138
  139
  140
  143
  144
  145
  146
  148
  149
  151
  152
  154
  155
  163
  167
  168
  169
  170
  171
  172
  173
  175
  177
  179
  181
  183
  184
  185
  186
  189
  190
  191
  192
  193
  195
  196
0 Samples buenos
152 Samples con errores de continuidad de tiempo
48 Samples descartados por falta de puntos


In [80]:
monitoringdf_tmp

NameError: name 'monitoringdf_tmp' is not defined

In [ ]:
list_samplesdf[0].columns

Index(['id_sample', 'date', 'date_time_end', 'date_time_init', 'end_idp',
       'end_lat', 'end_name', 'end_stop', 'id_link', 'ini_lat', 'init_idp',
       'init_name', 'init_stop', 'lap', 'link', 'time_travel', 'veh'],
      dtype='object')

In [ ]:
columnas_ord = ['id_sample', 'date', 'date_time_end', 'date_time_init', 'end_idp', 'end_name', 'end_stop', 'id_link', 'init_idp',
       'init_name', 'init_stop', 'lap', 'link', 'time_travel', 'veh', 'ini_lat',
       'end_lat']

In [ ]:
list_samplesdf[109][columnas_ord]

,id_sample,date,date_time_end,date_time_init,end_idp,end_name,end_stop,id_link,init_idp,init_name,init_stop,lap,link,time_travel,veh,ini_lat,end_lat
0,109,2023-03-01,2023-03-01 14:37:49,2023-03-01 14:04:39,13277666.0,primer stop,101,0,13271047.0,inicio,100,3.0,100:101,1990.0,SJ43,"(-13.5550828, -71.8559498)","(-13.5550275, -71.8570669)"
1,109,2023-03-01,2023-03-01 14:39:56,2023-03-01 14:37:49,13278087.0,segundo stop,102,1,13277666.0,primer stop,101,3.0,101:102,127.0,SJ43,"(-13.5550275, -71.8570669)","(-13.5532644, -71.8622568)"
2,109,2023-03-01,2023-03-01 14:41:46,2023-03-01 14:39:56,13278467.0,Ccollana,103,2,13278087.0,segundo stop,102,3.0,102:103,110.0,SJ43,"(-13.5532644, -71.8622568)","(-13.552325, -71.8672449)"
3,109,2023-03-01,2023-03-01 14:42:30,2023-03-01 14:41:46,13278612.0,Puente Huaccoto,104,3,13278467.0,Ccollana,103,3.0,103:104,44.0,SJ43,"(-13.552325, -71.8672449)","(-13.5516429, -71.8699255)"
4,109,2023-03-01,2023-03-01 14:43:59,2023-03-01 14:42:30,13278918.0,Kayra,105,4,13278612.0,Puente Huaccoto,104,3.0,104:105,89.0,SJ43,"(-13.5516429, -71.8699255)","(-13.5505811, -71.8741039)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,109,2023-03-01,2023-03-01 17:17:27,2023-03-01 17:17:15,13309590.0,Tapia,250,149,13309557.0,Posta,249,3.0,249:250,12.0,SJ43,"(-13.5490034, -71.8797405)","(-13.5492024, -71.8788978)"
150,109,2023-03-01,2023-03-01 17:18:16,2023-03-01 17:17:27,13309738.0,Forestal,251,150,13309590.0,Tapia,250,3.0,250:251,49.0,SJ43,"(-13.5492024, -71.8788978)","(-13.5499915, -71.8762344)"
151,109,2023-03-01,2023-03-01 17:18:40,2023-03-01 17:18:16,13309820.0,Kayra,252,151,13309738.0,Forestal,251,3.0,251:252,24.0,SJ43,"(-13.5499915, -71.8762344)","(-13.5505434, -71.8744531)"
152,109,2023-03-01,2023-03-01 17:20:18,2023-03-01 17:18:40,13310127.0,Puente,253,152,13309820.0,Kayra,252,3.0,252:253,98.0,SJ43,"(-13.5505434, -71.8744531)","(-13.5515743, -71.8703316)"


In [ ]:
list_samplesdf[109][columnas_ord].to_csv('sample_ida_vuela.csv', index=False)